In [26]:
import pygame
import math
import numpy as np
import sys

pygame.init()
screen = pygame.display.set_mode((800, 800))
clock = pygame.time.Clock()

# 정육면체 꼭짓점 정의
cube_vertex = np.array([
    [1, 1, 1],
    [1, 1, -1],
    [1, -1, 1],
    [1, -1, -1],
    [-1, 1, 1],
    [-1, 1, -1],
    [-1, -1, 1],
    [-1, -1, -1]
])

# 정육면체의 간선 정의
edges = [
    (0, 1), (0, 2), (0, 4),
    (1, 3), (1, 5),
    (2, 3), (2, 6),
    (3, 7),
    (4, 5), (4, 6),
    (5, 7),
    (6, 7)
]

# 3D → 2D 투영 함수
def project(points):
    z = points[2] + 5
    factor = 500 / z
    x = points[0] * factor + 400
    y = points[1] * factor + 400
    return (int(x), int(y))

# X축 회전
def x_rotate(angle):
    cos_theta = math.cos(angle)
    sin_theta = math.sin(angle)
    return np.array([
        [1, 0, 0],
        [0, cos_theta, -sin_theta],
        [0, sin_theta, cos_theta]
    ])

# Y축 회전
def y_rotate(angle):
    cos_theta = math.cos(angle)
    sin_theta = math.sin(angle)
    return np.array([
        [cos_theta, 0, sin_theta],
        [0, 1, 0],
        [-sin_theta, 0, cos_theta]
    ])

# Z축 회전
def z_rotate(angle):
    cos_theta = math.cos(angle)
    sin_theta = math.sin(angle)
    return np.array([
        [cos_theta, -sin_theta, 0],
        [sin_theta, cos_theta, 0],
        [0, 0, 1]
    ])

# 각도 초기화
angle = 0

# 메인 루프
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    screen.fill((0, 0, 0))

    # 회전 행렬 생성 (X, Y, Z축 모두 회전)
    rx = x_rotate(angle)
    ry = y_rotate(angle)
    rz = z_rotate(angle)

    # 회전 적용
    rotated_vertex = [rz @ ry @ rx @ v for v in cube_vertex]

    # 2D로 투영
    projected_vertex = [project(v) for v in rotated_vertex]

    # 간선 그리기
    for edge in edges:
        pygame.draw.line(screen, (255, 255, 255),
                         projected_vertex[edge[0]],
                         projected_vertex[edge[1]], 2)

    pygame.display.flip()
    clock.tick(60)

    # 각도 증가
    angle += 0.03


SystemExit: 

C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
